In [46]:
import math
from math import log
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
from importlib import reload

reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s

print()

In [107]:
def max_min_window(a: List[int]) -> List[int]:
    "For every window size return the maximum of the minimums."

    def bracket(b: List[int], it: Iterable[int]) -> List[int]:
        s = []
        for i in it:
            while s and a[i] <= a[s[-1]]:
                s.pop()
            if s:
                b[i] = s[-1]
            s.append(i)
        return b

    n = len(a)
    left = bracket([-1] * n, range(n))
    right = bracket([n] * n, reversed(range(n)))

    print(left)
    print(right)

    wnd = [0] * n
    for e, l, r in zip(a, left, right):
        wnd[r - l - 2] = max(wnd[r - l - 2], e)

    print(wnd)

    for i in reversed(range(n - 1)):
        wnd[i] = max(wnd[i], wnd[i + 1])

    print(wnd)

    return wnd

In [108]:
max_min_window([10, 20, 30, 50, 10, 70, 30])

[-1, 0, 1, 2, -1, 4, 4]
[7, 4, 4, 4, 7, 6, 7]
[70, 30, 20, 0, 0, 0, 10]
[70, 30, 20, 10, 10, 10, 10]


[70, 30, 20, 10, 10, 10, 10]